In [16]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from custom_losses import *
from custom_metrics import *

from models import build_unet, build_w_net
import os
import cv2

In [17]:
# Define paths to your dataset
image_folder = 'DRIVE/training/images'
label_folder = 'DRIVE/training/1st_manual'

# Get list of all files in the image and label directories
image_files = sorted(os.listdir(image_folder))
label_files = sorted(os.listdir(label_folder))



In [18]:
# Function to preprocess images and labels
def preprocess(image_path, label_path):
    # Convert the TensorFlow Tensor object to a string
    image_path = image_path.decode('utf-8')
    label_path = label_path.decode('utf-8')
    # Load image and label
    image = cv2.imread(image_path)
    label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
    # Resize images and labels
    image_resized = cv2.resize(image, (512, 512))
    label_resized = cv2.resize(label, (512, 512))
    
    # Normalize images and labels
    image_normalized = image_resized.astype(np.float32) / 255.0
    label_normalized = label_resized.astype(np.float32) / 255.0
    label_normalized = label_normalized[:, :, np.newaxis]
    return image_normalized, label_normalized



In [19]:
# Function to load the dataset into tf.data.Dataset
def load_dataset(image_folder, label_folder):
    # Create file paths
    image_paths = [os.path.join(image_folder, f) for f in image_files]
    label_paths = [os.path.join(label_folder, f) for f in label_files]
    # Create a TensorFlow dataset from the file paths
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
    
    # Map the preprocessing function to each element in the dataset
    dataset = dataset.map(lambda img_path, lbl_path: tf.numpy_function(
                          func=preprocess, inp=[img_path, lbl_path], 
                          Tout=[tf.float32, tf.float32]))
    
    # Shuffle and batch the dataset
    dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(4)  # Set batch size to 32, modify as needed
    
    return dataset



In [20]:
# Load the dataset
dataset = load_dataset(image_folder, label_folder)

In [21]:
for batch in dataset:
    image, label = batch
    for i in label:
        print(i.shape)

(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)
(512, 512, 1)


In [22]:
num_epoch = 4000
step = 50
# Define the cosine annealing learning rate scheduler
def cosine_annealing_schedule(epoch, lr):
    initial_lr = lr  # Use the current learning rate as the initial rate
    if lr%step == 0:
        lr = 0.5 * initial_lr * (1 + np.cos(np.pi * (epoch) / num_epoch))
    return lr

# Create the LearningRateScheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(cosine_annealing_schedule)

In [23]:
wnet = build_w_net((512, 512, 3))  # Input size is 128x128 grayscale image
wnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_20 (Conv2D)             (None, 512, 512, 8)  224         ['input_2[0][0]']                
                                                                                                  
 conv2d_21 (Conv2D)             (None, 512, 512, 8)  584         ['conv2d_20[0][0]']              
                                                                                                  
 max_pooling2d_4 (MaxPooling2D)  (None, 256, 256, 8)  0          ['conv2d_21[0][0]']        

In [24]:

wnet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1E-2), loss={"out1":DiceLoss(), "out2":DiceLoss()}, metrics=[iou_metric])


In [25]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, verbose=1, mode='min')
wnet.fit(x=dataset, epochs=200, verbose=2, callbacks=[early_stopping])

Epoch 1/200
5/5 - 10s - loss: 1.4571 - out1_loss: 0.7264 - out2_loss: 0.7307 - out1_iou_metric: 0.0779 - out2_iou_metric: 0.0791 - 10s/epoch - 2s/step
Epoch 2/200
5/5 - 2s - loss: 1.3863 - out1_loss: 0.6910 - out2_loss: 0.6953 - out1_iou_metric: 0.0725 - out2_iou_metric: 0.0770 - 2s/epoch - 337ms/step
Epoch 3/200
5/5 - 2s - loss: 1.3125 - out1_loss: 0.6321 - out2_loss: 0.6803 - out1_iou_metric: 0.0961 - out2_iou_metric: 0.0821 - 2s/epoch - 329ms/step
Epoch 4/200
5/5 - 2s - loss: 1.2687 - out1_loss: 0.6206 - out2_loss: 0.6482 - out1_iou_metric: 0.1013 - out2_iou_metric: 0.0906 - 2s/epoch - 330ms/step
Epoch 5/200
5/5 - 2s - loss: 1.2973 - out1_loss: 0.6182 - out2_loss: 0.6790 - out1_iou_metric: 0.1035 - out2_iou_metric: 0.0887 - 2s/epoch - 328ms/step
Epoch 6/200
5/5 - 2s - loss: 1.2429 - out1_loss: 0.6184 - out2_loss: 0.6245 - out1_iou_metric: 0.1052 - out2_iou_metric: 0.1055 - 2s/epoch - 331ms/step
Epoch 7/200
5/5 - 2s - loss: 1.2411 - out1_loss: 0.6167 - out2_loss: 0.6245 - out1_iou_me

KeyboardInterrupt: 